# GATO Pick & Place (New Single-Input API)

This notebook demonstrates using the new single-input BSQP flow with automatic external wrench estimation and GPU-side batching.

In [ ]:
import numpy as np
import pinocchio as pin
import sys
sys.path.append('python')
from bsqp.interface import BSQP

# Config
model_path = 'iiwa_description/iiwa14.urdf'
batch_size = 32
N = 16
dt = 0.02

# Instantiate solver (single-input API)
solver = BSQP(model_path=model_path, batch_size=batch_size, N=N, dt=dt,
              max_sqp_iters=10, kkt_tol=1e-4, max_pcg_iters=200, pcg_tol=1e-3,
              solve_ratio=1.0, mu=1.0, q_cost=5e-2, qd_cost=2e-3, u_cost=2e-7,
              N_cost=10.0, q_lim_cost=1e-5, vel_lim_cost=1e-3, ctrl_lim_cost=1e-4, rho=1e-5)

nq = solver.model.nq
nv = solver.model.nv
nx = nq + nv
nu = nv

def make_ee_ref(goal, N):
    g = np.concatenate([goal, np.zeros(3, dtype=np.float32)]).astype(np.float32)
    return np.tile(g, N)

# Initial state (home)
q0 = np.zeros(nq, dtype=np.float32)
v0 = np.zeros(nv, dtype=np.float32)
x = np.concatenate([q0, v0])
u_prev = np.zeros(nu, dtype=np.float32)

# Pick & place waypoints (xyz in meters)
goals = [
    np.array([0.5, 0.0, 0.4], dtype=np.float32),
    np.array([0.5, 0.2, 0.2], dtype=np.float32),
    np.array([0.6, -0.2, 0.2], dtype=np.float32),
    np.array([0.5, 0.0, 0.5], dtype=np.float32),
]

traj = [x.copy()]

for gi, goal in enumerate(goals):
    ee_ref = make_ee_ref(goal, N)

    stats = solver.solve(x, ee_ref)
    XU_best = solver.select_best_and_copy(np.zeros(14), np.zeros(7), x, dt)

    # Run a few steps reusing the GPU batch
    for t in range(10):
        stats = solver.solve_existing_with_ref(x, ee_ref)
        XU_best = solver.select_best_and_copy(np.zeros(14), np.zeros(7), x, dt) 
        u = XU_best[nx:nx+nu]
        x_next_B = solver.sim_forward(x, u, dt)
        x_next = np.asarray(x_next_B)[0]  # take the first row
        traj.append(x_next.copy())
        u_prev = u.copy()
        x = x_next

print('Done. Trajectory length:', len(traj))
print(traj)


T : f
Done. Trajectory length: 41
[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32), array([-1.5567016e-04,  7.5538788e-04,  6.4459673e-05,  1.7671493e-03,
        1.7723581e-03,  9.2147192e-04, -1.6943746e-03, -1.5567016e-02,
        7.5538792e-02,  6.4459676e-03,  1.7671493e-01,  1.7723581e-01,
        9.2147194e-02, -1.6943747e-01], dtype=float32), array([-0.00066267,  0.0030577 ,  0.00036327,  0.0071655 ,  0.00709391,
        0.00381809, -0.00686976, -0.03513255,  0.15469202,  0.02343508,
        0.3631202 ,  0.35491934,  0.19751446, -0.34810117], dtype=float32), array([-0.00142788,  0.00701102,  0.00083651,  0.01649478,  0.0163106 ,
        0.00948402, -0.01631527, -0.0413885 ,  0.24064052,  0.02388893,
        0.5698075 ,  0.56674933,  0.36907881, -0.5964496 ], dtype=float32), array([-0.00233009,  0.01277861,  0.00137376,  0.03021547,  0.0300012 ,
        0.01886208, -0.0311216 , -0.04883272,  0.33611792,  0.02983618,
        0.8022616 ,  0.802